In [1]:
import os
from tkinter import filedialog
from tkinter import *
import numpy as np
import re
import random as rdm

In [2]:
# Get scene images directory
if not os.path.isdir(os.getcwd()+'/scene_images'):
    root = Tk()
    root.withdraw()
    scene_dir = filedialog.askdirectory()
else:
    scene_dir = os.getcwd()+'/scene_images'
    
# Get morphed scene images directory
if not os.path.isdir(os.getcwd()+'/morphed_images'):
    root = Tk()
    root.withdraw()
    morphed_dir = filedialog.askdirectory()
else:
    morphed_dir = os.getcwd()+'/morphed_images'

In [3]:
morphscenes = os.listdir(morphed_dir)
allScenes = {}
for scene in morphscenes:
    if scene.endswith('.png'):
        cur_info = np.array(scene.split('_'))
        cur_scene = '_'.join(cur_info[0:len(cur_info)-3]) + '.png'
        cur_xy = re.findall('[0-9]+', cur_info[3])
        cur_type = cur_info[2]
        if cur_scene not in allScenes.keys():
            allScenes[cur_scene] = {}
        allScenes[cur_scene][cur_type] = {'morphed':scene,'x':cur_xy[0],'y':cur_xy[1]}
            

In [30]:
# remove unnecessary scenes from scene directory
scenes = list(allScenes.keys())
scenesindir = os.listdir(scene_dir)
for scene in scenesindir:
    if scene not in scenes:
        os.remove(scene_dir + '/' + scene)
        
# sanity check
scenes = np.sort(list(allScenes.keys()))
scenesindir = np.sort(os.listdir(scene_dir))
if not np.array_equal(scenes, scenesindir):
    raise ValueError('Scenes are missing in scene directory or morphed scene directory')

In [110]:
# create two conditions, counterbalanced red/green across participants
scene_n = len(allScenes.keys())
# assign randomized red/green conditions
redgreen = np.concatenate((np.zeros(int(scene_n/2), dtype=bool), np.ones(int(scene_n/2), dtype=bool)))
rdm.shuffle(redgreen)
# randomize order of scenes 
scene_order = list(allScenes.keys())
rdm.shuffle(scene_order)

In [111]:
# write conditions
# write first condition
with open('input01.csv', 'w') as f:
    f.write('x,y,first_image,second_image,rad,type\n')
    for idx in range(len(scene_order)):
        if redgreen[idx]:
            cur_type = 'red'
        else:
            cur_type = 'green'
        cur_scene = scene_order[idx]
        cur_x = allScenes[cur_scene][cur_type]['x']
        cur_y = allScenes[cur_scene][cur_type]['y']
        cur_second_img = allScenes[cur_scene][cur_type]['morphed']
        f.write(f'{cur_x},{cur_y},{cur_scene},{cur_second_img},43.5,{cur_type}\n')
# write second condition
with open('input02.csv', 'w') as f:
    f.write('x,y,first_image,second_image,rad,type\n')
    for idx in range(len(scene_order)):
        if not redgreen[idx]:
            cur_type = 'red'
        else:
            cur_type = 'green'
        cur_scene = scene_order[idx]
        cur_x = allScenes[cur_scene][cur_type]['x']
        cur_y = allScenes[cur_scene][cur_type]['y']
        cur_second_img = allScenes[cur_scene][cur_type]['morphed']
        f.write(f'{cur_x},{cur_y},{cur_scene},{cur_second_img},43.5,{cur_type}\n')